# Inferência Bayseana

- Nome: Henrique Tibério Brandão Vieira Augusto
- Matrícula: 2186330010
- Turma: 2021-1

**INSTRUÇÕES**:

- **Mantenha a estrutura de enunciados e células de resposta apresentada neste modelo.**
- **Todas as conclusões devem estar fundamentadas com experimentos e análises de dados que corroborem os aspectos afirmados.**

## Exercício 1

No colab "2 - Inferência", no exemplo "Update Bayesiano com distribuição Beta", foi apresentado um procedimento simplificado de inferência Bayesiana para a distribuição beta. Neste update, os parâmetros $\alpha, \beta$ estão relacionados ao grau de crença sobre a hipótese. Diferentes parametrizações geram diferentes diferentes distribuições que se assemelham às gaussianas, às uniformes e, inclusive, distribuições assimétricas.

No exemplo em questão, o cenário simula a jogada de uma moeda, e, com a coleta de dados a respeito das jogadas, deseja-se tirar conclusões a respeito da moeda. Neste caso, tem-se um _a priori_ modelado por uma distribuição beta e o modelo relativo aos dados (as jogadas da moeda) é implicitamente representado por uma distribuição de Bernoulli. E, por esta razão, sabe-se que o _a posteriori_ também é conhecido, sendo também uma distribuição beta. O conceito utilizado neste caso, é o de **a priori conjugado**: certas configurações de _a priori_ e verossimilhança determinam a forma do _a posteriori_.

**Qual o impacto do _a priori_ e dos parâmetros na inferência?** Faça um estudo de caso a respeito da simluação de diferentes moedas onde o sucesso do experimento de Bernoulli representa a saída de cara na moeda. Analise diferentes valores para o _ground-truth_ $p$ da moeda, diferentes valores para o número de jogadas, e diferentes valores para $\alpha, \beta$. Explore _a prioris_ simétricos, assimétricos, com maiores ou menores magnitudes. Disserte a respeito do impacto desses parâmetros para os resultados obtidos no _a posteriori_.

## Exercício 2

No colab "2 - Inferência", no tópico "Exemplo da altura dos programdores Python" foi apresentada uma modelagem para inferência que usava de referência a altura média da população do planeta como _a priori_ para a inferência da média do _a posteriori_. Pra a inferência da variância da distribuição, foi utilizado um _a priori_ não informado.

Implemente um cenário semelhante, onde a amostra de alturas coletadas seja dada por uma Gaussiana de média 180 e variância 9. O tamanho da amostra deve ser a altura de 100 programadores. Faça uma inferência em grid da média e variância do _a posteriori_ considerando tanto _a priori_'s desinformados para média e variância, quanto _a priori_'s a partir das informações prévias de média e variância da altura das pessoas do planeta. **Analise as diferenças entre os _a posteriori_ obitos**.

## Exercício 3

Implemente um modelo de VAE Gaussiano para uma base de escolha do grupo, que seja diferente da apresentada na sala de aula. Treine este modelo em um latente de tamanho arbitrário (de dimensionalidade 3 ou maior). Exiba as propriedades do modelo treinado para geração de amostras com a combinação das principais características atreladas aos agrupamentos do espaço latente. 

Para tal, utilize o procedimento de encontrar os vetores de média dos dois tipos de dados sobre os quais deseja-se gerar a amostra combinada e gere o vetor apropriado para o decoder que represente combinações das características em questão.

Exiba alguns exemplos gerados a partir das combinações escolhidas.

## Exercício 4

Implemente uma arquitetura de rede neural profunda Bayesiana com a biblioteca Pyro em algum dataset de escolha do grupo. Avalie a robustez da rede neural bayesiana usando a estratégia de amostragem de redes neurais apresentada em sala de aula.

Disserte a respeito do desempenho do modelo treinado em um dataset de validação atrelado ao dataset de treinamento. Avalie também o comportamento da rede neural para padrões de dados diferentes dos dados utilizados para treinamento. Desde dados levemente diferentes a dados completamente diferentes.

Neste contexto, disserte a respeito das ocasiões onde a rede neural chegou a uma classificação quando não deveria classificar o dado e vice-versa, quando a rede neural não classificou dados que deveria ser capaz de classificar. Através da estratégia de composição de distribuições para as previsões da rede neural, também considere nas avaliações diferentes limiares de incerteza para considerar que houve uma predição ou não da rede neural Bayesiana para o dado em questão.